# Loading and saving models

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import os 

processor = WhisperProcessor.from_pretrained("bofenghuang/whisper-large-v2-cv11-german")
model = WhisperForConditionalGeneration.from_pretrained("bofenghuang/whisper-large-v2-cv11-german")
model.config.forced_decoder_ids = None

# Assuming `model` is your trained model and `processor` is your processor
save_directory = "models/whisper-large-v2-cv11-german"

# Create output directory if needed
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Save the model
model.save_pretrained(save_directory)

# Save the processor/tokenizer
processor.save_pretrained(save_directory)

# With manual setting of language:

In [3]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor, WhisperTokenizer, pipeline
import torch
import librosa

feature_extractor = WhisperFeatureExtractor.from_pretrained("models/whisper-large-v2-cv11-german")
tokenizer = WhisperTokenizer.from_pretrained("models/whisper-large-v2-cv11-german", language="german", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained("models/whisper-large-v2-cv11-german")
forced_decoder_ids = tokenizer.get_decoder_prompt_ids(language="de", task="transcribe")

if torch.cuda.is_available():
    device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

asr_pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    chunk_length_s=30,
    stride_length_s=10,
    device=device,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.14it/s]


In [3]:
from pydub import AudioSegment

# Specify the path to your local audio file
audio_file = "demoFiles/InterviewMuenchen.mp3"

# Load audio file
audio = AudioSegment.from_mp3(audio_file)

# Slice the first minute
one_minute_audio = audio[:60000]  # pydub works in milliseconds

# Save to a temporary file
temp_file = "temp.mp3"
one_minute_audio.export(temp_file, format="mp3")

# Use the temporary file for prediction
prediction = asr_pipe(temp_file, batch_size=1, generate_kwargs={"forced_decoder_ids": forced_decoder_ids})

print(prediction['text'])

FileNotFoundError: [WinError 2] The system cannot find the file specified